## Load the Tweets Dataset

In [20]:
import nltk
import matplotlib.pyplot as plt
import contractions
from nltk.stem.wordnet import WordNetLemmatizer
import re
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer


In [12]:
import pandas as pd
pd.set_option("display.max_colwidth", 200)

In [15]:
# tweets = pd.read_pickle("cleaned_tweets_v1.pkl")
tweets = pd.read_csv("tweets.csv")
tweets.head(5)

,id,label,tweet
0,1,0,#fingerprint #Pregnancy Test https://goo.gl/h1MfQV #android #apps #beautiful #cute #health #igers #iphoneonly #iphonesia #iphone
1,2,0,Finally a transparant silicon case ^^ Thanks to my uncle :) #yay #Sony #Xperia #S #sonyexperias… http://instagram.com/p/YGEt5JC6JM/
2,3,0,We love this! Would you go? #talk #makememories #unplug #relax #iphone #smartphone #wifi #connect... http://fb.me/6N3LsUpCu
3,4,0,I'm wired I know I'm George I was made that way ;) #iphone #cute #daventry #home http://instagr.am/p/Li_5_ujS4k/
4,5,1,What amazing service! Apple won't even talk to me about a question I have unless I pay them $19.95 for their stupid support!


In [4]:
tweets.shape

(7920, 4)

In [16]:
# LEt's map 0 as 1, and 1 as 0
tweets['label'] = tweets['label'].map({0: 1, 1: 0})
tweets.head()

,id,label,tweet
0,1,1,#fingerprint #Pregnancy Test https://goo.gl/h1MfQV #android #apps #beautiful #cute #health #igers #iphoneonly #iphonesia #iphone
1,2,1,Finally a transparant silicon case ^^ Thanks to my uncle :) #yay #Sony #Xperia #S #sonyexperias… http://instagram.com/p/YGEt5JC6JM/
2,3,1,We love this! Would you go? #talk #makememories #unplug #relax #iphone #smartphone #wifi #connect... http://fb.me/6N3LsUpCu
3,4,1,I'm wired I know I'm George I was made that way ;) #iphone #cute #daventry #home http://instagr.am/p/Li_5_ujS4k/
4,5,0,What amazing service! Apple won't even talk to me about a question I have unless I pay them $19.95 for their stupid support!


In [17]:
# drop the id column
tweets.drop("id", axis=1, inplace=True)

In [18]:
tweets.head()

,label,tweet
0,1,#fingerprint #Pregnancy Test https://goo.gl/h1MfQV #android #apps #beautiful #cute #health #igers #iphoneonly #iphonesia #iphone
1,1,Finally a transparant silicon case ^^ Thanks to my uncle :) #yay #Sony #Xperia #S #sonyexperias… http://instagram.com/p/YGEt5JC6JM/
2,1,We love this! Would you go? #talk #makememories #unplug #relax #iphone #smartphone #wifi #connect... http://fb.me/6N3LsUpCu
3,1,I'm wired I know I'm George I was made that way ;) #iphone #cute #daventry #home http://instagr.am/p/Li_5_ujS4k/
4,0,What amazing service! Apple won't even talk to me about a question I have unless I pay them $19.95 for their stupid support!


In [19]:
def tweet_cleaner(raw_tweet):
    '''
    This function cleans the raw tweet
    '''
    #Substituting contractions
    cleaned_tweet = contractions.fix(raw_tweet)

    # User-mentions Removal
    cleaned_tweet = re.sub("@[A-Za-z0-9]+", "", cleaned_tweet)

    # Hashtag Removal
    cleaned_tweet = re.sub("#", "", cleaned_tweet)

    #Hyperlink Removal
    cleaned_tweet = re.sub(r"http\S+","", cleaned_tweet)

    # Punctuation, Special Characters and digits Removal (Retaining only the alphabets)
    cleaned_tweet = re.sub(r"[^a-zA-Z]", " " , cleaned_tweet )
    
    # convert the tweet into lowercase & get rid of any leading or trailing spaces
    cleaned_tweet = cleaned_tweet.lower().strip()  

    # Retain only those token which have length > 2 characters
    cleaned_tweet = [token for token in cleaned_tweet.split() if len(token)>2]
    
    new_sent = ''
    lemmatizer = WordNetLemmatizer()
    for token in cleaned_tweet:
        new_sent = new_sent + lemmatizer.lemmatize(token) + ' '
    
    return new_sent.strip()

In [ ]:
# Apply this cleaner function on ALL the tweets
# tweets["cleaned_tweets"] = tweets["tweet"].apply(tweet_cleaner)
# tweets.head()

## CREATE PIPELINE WHICH INCLUDES THE DATA CLEANER FUNCTION, ALONGWITH THE TF-IDF & LOGISTIC REGRESSION

In [21]:
from sklearn.base import BaseEstimator, TransformerMixin

class TextCleanerTransformer(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        return [tweet_cleaner(text) for text in X]


# Model Building using Pipeline

In [22]:
TFIDF = TfidfVectorizer(max_df=0.5, min_df=10, max_features=300)
LR = LogisticRegression(solver='liblinear', class_weight='balanced', penalty='l2', C=0.5)

model_pipeline = Pipeline([('cleaner', TextCleanerTransformer()), \
                           ("TFIDF", TFIDF), ("LR", LR)])

In [31]:
X = tweets['tweet']
y = tweets['label']
model_pipeline.fit(X,y)

Pipeline(steps=[('cleaner', TextCleanerTransformer()),
                ('TFIDF',
                 TfidfVectorizer(max_df=0.5, max_features=300, min_df=10)),
                ('LR',
                 LogisticRegression(C=0.5, class_weight='balanced',
                                    solver='liblinear'))])

In [35]:
# Save the trained pipeline
import joblib
joblib.dump(model_pipeline, 'model_pipeline.pkl')

['model_pipeline.pkl']

In [36]:
model_pipeline = joblib.load('model_pipeline.pkl')

In [37]:
model_pipeline.predict(X)

array([1, 1, 1, ..., 0, 1, 1], dtype=int64)